MIMIC_Sepsis
=================

# 1 Preparation

To run this document the following requirements must be satisfied:

- Implement the database mimic in **PostgreSQL** and start it. The instruction can be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres). (The name of this environment should be **mimiciv**)
- generate useful abstractions of raw MIMIC-IV data. The instruction be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/concepts_postgres) 



To install all the required libraries, uncomment following and run:

In [46]:
# !pip install -r requirements.txt

Run the following cell to connect to the database.

In [47]:
%load_ext autoreload
%autoreload 2

import psycopg2
from psycopg2 import sql
import csv
import pandas as pd
import numpy as np
import os
import shutil
import csv
from datetime import timedelta
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor

# implement the username, password and database name
conn = psycopg2.connect(host='', user='', password='', database='mimiciv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2 Extract selected data from the original database 

According to the paper we need to extract the **state space** and **action space** respectively from the mimiciv database. The table **mimic4 itemid.csv** lists all the items required.

***Uncomment the following line if you first time run the code***

In [48]:
# # uncomment the following line if you first time run the code

# # Read the SQL file

# try:
#     with open('sql/select_patients_cohort.sql', 'r') as file0:
#         sql_script_select_patients_cohort = file0.read()
        
#     with open('sql/state_from_chartevents.sql', 'r') as file1:
#         sql_script_state = file1.read()

#     with open('sql/action_from_inputevents.sql', 'r') as file2:
#         sql_script_action = file2.read()

#     # Execute the SQL script
#     cursor = conn.cursor()
    
#     cursor.execute(sql.SQL(sql_script_select_patients_cohort))
#     cursor.execute(sql.SQL(sql_script_state))
#     cursor.execute(sql.SQL(sql_script_action))

#     conn.commit()
#     cursor.close()
    
# except (Exception, psycopg2.DatabaseError) as error:
#     print("Error executing SQL statement:", error)

Get the number of stay_ids

In [49]:
with conn.cursor() as cursor:
    command = "SELECT distinct stay_id FROM mimiciv_derived.sepsis_patients_cohort;"
    cursor.execute(command)   
    result = cursor.fetchall()
    stay_ids= [row[0] for row in result]
    num_stay_ids = len(stay_ids)
    print('Number of stay_ids: ' + str(num_stay_ids))
    cursor.close()

Number of stay_ids: 6669


# 3 Data transfer

## 3.1 Data transfer of State Space
We transfer the data of State Space from Postgresql to csv.

In [50]:
from python.data_preprocessing.data_transfer import data_transfer_state
data_transfer_state(conn, num_stay_ids, threshold = 1000)

output:Heartrate.csv                           	number of stay_id:6669
output:ABPs.csv                                	number of stay_id:2129
output:NBPs.csv                                	number of stay_id:6632
output:ABPd.csv                                	number of stay_id:2131
output:NBPd.csv                                	number of stay_id:6632
output:ABPm.csv                                	number of stay_id:2168
output:NBPm.csv                                	number of stay_id:6632
output:RespiratoryRate.csv                     	number of stay_id:6669
output:TemperatureF.csv                        	number of stay_id:6576
output:TemperatureC.csv                        	number of stay_id:832
output:PH_A.csv                                	number of stay_id:3663
output:PH_V.csv                                	number of stay_id:3184
output:ABE.csv                                 	number of stay_id:3645
output:Hematocrit_serum.csv                    	number of stay_id:6596
output:

## 3.2 Data transfer of Action Space

In *3.2.1* we transfer data of following actions from Postgresql to csv:

 - IV fluid bolus
   - NaCl_0.9%
   - Dextrose_5%


After calculation and comparison, in *3.2.2* we will omit the two insignificant vasopressors: Dobutamine and Milrinone.

So that in *3.2.3* we can directly obtain a vasopressors_norepinephrine_equivalent_dose based on "Vasopressor dose equivalence: A scoping review and suggested formula" by Goradia et al. 2020.

### 3.2.1 Data transfer of Action Space for *IV fluid bolus*

 - IV fluid bolus
   - NaCl_0.9%
   - Dextrose_5%

In [51]:
from python.data_preprocessing.data_transfer import data_transfer_action_IV_fluid_bolus
data_transfer_action_IV_fluid_bolus(conn)

output action (IV_fluid_bolus):	NaCl_0_9%.csv
output action (IV_fluid_bolus):	Dextrose_5%.csv


### 3.2.2 Data transfer of Action Space for *Vasopressors*

we directly obtain `vasopressors_norepinephrine_equivalent_dose` 

from `mimiciv_derived.norepinephrine_equivalent_dose` 

based on *"Vasopressor dose equivalence: A scoping review and suggested formula" by Goradia et al. 2020*.

In [53]:
from python.data_preprocessing.data_transfer import data_transfer_action_vasopressors_equivalent_dose
data_transfer_action_vasopressors_equivalent_dose(conn)

output action (vasopressors): vasopressors_norepinephrine_equivalent_dose.csv


# 4 Hourly Sample

## 4.1 Hourly Sample on State Space

### 4.1.1 Define function hourly_sample_state(selected_id)